In [40]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# download wikipage
wikipage = "https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/04-01-2020.csv"
result = requests.get(wikipage)

# if successful parse the download into a BeautifulSoup object, which allows easy manipulation 
if result.status_code == 200:
    soup = BeautifulSoup(result.content, "html.parser")
    
# find the object with HTML class wibitable sortable
table = soup.find('table',{'class':'js-csv-data csv-data js-file-line-container'})

# loop through all the rows and pull the text
new_table = []
for row in table.find_all('tr')[1:]:
    column_marker = 0
    columns = row.find_all('td')
    new_table.append([column.get_text() for column in columns])
    
df = pd.DataFrame(new_table)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,,45001,Abbeville,South Carolina,US,2020-04-01 21:58:49,34.22333378,-82.46170658,4,0,0,0,"Abbeville, South Carolina, US"
1,,22001,Acadia,Louisiana,US,2020-04-01 21:58:49,30.295064899999996,-92.41419698,47,1,0,0,"Acadia, Louisiana, US"
2,,51001,Accomack,Virginia,US,2020-04-01 21:58:49,37.76707161,-75.63234615,7,0,0,0,"Accomack, Virginia, US"
3,,16001,Ada,Idaho,US,2020-04-01 21:58:49,43.4526575,-116.24155159999998,195,3,0,0,"Ada, Idaho, US"
4,,19001,Adair,Iowa,US,2020-04-01 21:58:49,41.33075609,-94.47105874,1,0,0,0,"Adair, Iowa, US"


In [41]:
# add headers tp dataframe
df.columns = ['Index','FIPS','Admin2','Province_State','Country_Region',
              'Last_Update','Lat','Long_','Confirmed','Deaths','Recovered','Active','Combined_Key']
df = df.drop(columns="Index")
df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001,Abbeville,South Carolina,US,2020-04-01 21:58:49,34.22333378,-82.46170658,4,0,0,0,"Abbeville, South Carolina, US"
1,22001,Acadia,Louisiana,US,2020-04-01 21:58:49,30.295064899999996,-92.41419698,47,1,0,0,"Acadia, Louisiana, US"
2,51001,Accomack,Virginia,US,2020-04-01 21:58:49,37.76707161,-75.63234615,7,0,0,0,"Accomack, Virginia, US"
3,16001,Ada,Idaho,US,2020-04-01 21:58:49,43.4526575,-116.24155159999998,195,3,0,0,"Ada, Idaho, US"
4,19001,Adair,Iowa,US,2020-04-01 21:58:49,41.33075609,-94.47105874,1,0,0,0,"Adair, Iowa, US"


In [42]:
df['Total']= df['Confirmed']+df['Deaths']+df['Recovered']+df['Active']

In [44]:
import plotly.graph_objects as go

df['text'] = ('Confirmed: '+ df['Confirmed'].astype(str) + '\n' + 'Deaths:'+ df['Deaths'].astype(str) 
        + '\n Recovered: ' + df['Recovered'].astype(str) + '\n Active: ' + df['Active'].astype(str))

fig = go.Figure(data=go.Scattergeo(
        lon = df['Long_'],
        lat = df['Lat'],
        mode = 'markers',
        text = df['text']
        ))

fig.update_layout(
        title = 'Corona Virus Cases in the US)',
        geo_scope='world',
        hoverlabel=dict(
            bgcolor="white", 
            font_size=16, 
            font_family="Rockwell")
    )
fig.show()